<a href="https://colab.research.google.com/github/arnabanupam/UNET/blob/master/UnetModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import os
import skimage.io as io
import skimage.transform as trans
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, Dropout,  concatenate, UpSampling2D
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras import losses as l
from keras import optimizers

Using TensorFlow backend.


In [0]:
smooth=1
def dice_coef(y_true, y_pred):
    y_true_f = keras.flatten(y_true)
    y_pred_f = keras.flatten(y_pred)
    intersection = keras.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (keras.sum(y_true_f*y_true_f) + keras.sum(y_pred_f*y_pred_f) + smooth)

In [0]:
def unet(pretrained_weights = None,input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)


In [0]:
 model = Model(input = inputs, output = conv10)


In [0]:
 model.compile(optimizer = Adam(lr = 1e-4), loss = l.binary_crossentropy, metrics = ['dice_coef'])

In [0]:
if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

# PREPROCESSING

In [1]:
import cv2
import h5py
import glob
import SimpleITK as sitk

from keras import callbacks as K
from keras import backend as kb
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from keras.engine import Model
import tensorlayer as tl

Using TensorFlow backend.


RuntimeError: ignored

In [2]:
!pip install --upgrade tensorflow

     |████████████████████████████████| 109.2MB 205kB/s 
     |████████████████████████████████| 3.2MB 19.6MB/s 
  Found existing installation: tensorboard 1.13.1
    Uninstalling tensorboard-1.13.1:
      Successfully uninstalled tensorboard-1.13.1
  Found existing installation: tensorflow 1.14.0rc1
    Uninstalling tensorflow-1.14.0rc1:
      Successfully uninstalled tensorflow-1.14.0rc1


In [0]:
Dataset = 'HGG'p
p = os.listdir(Dataset)

In [0]:
F1=[glob.glob(Dataset+ '/' + p1+('/*flair.nii*'))[0] for p1 in p[:]]
T1=[glob.glob(Dataset+ '/' + p1+('/*t1.nii*'))[0] for p1 in p[:]]
TCE=[glob.glob(Dataset+ '/' + p1+('/*t1ce.nii*'))[0] for p1 in p[:]]
T2=[glob.glob(Dataset+ '/' + p1+('/*t2.nii*'))[0] for p1 in p[:]]
SG=[glob.glob(Dataset+ '/' + p1+('/*seg.nii*'))[0] for p1 in p[:]]


In [0]:
data_F1=[]
data_T1=[]
data_TCE=[]
data_T2=[]
data_SG=[]
data_X=[]
data_Y=[]

In [0]:
for i in range(len(F1)):
  
    X = sitk.ReadImage(F1[i])  
    img = sitk.GetArrayFromImage(X)
    img=np.asarray(img,'float32')
#    print(np.shape(img))
    img=cv2.resize(img,(240,240))
    img=np.transpose(img,(1,2,0))
    img=cv2.resize(img,(240,240))
    img=np.transpose(img,(2,0,1))
    img=np.asarray(img,'int')
    img=np.asarray(img,'float32')

    img1=img
    #    print(np.shape(img))
    for j in range(img.shape[0]):        
        if (np.sum(img1[j])>0):
            data_F1.append(img[j])


    X = sitk.ReadImage(T1[i])
    img = sitk.GetArrayFromImage(X)
    img=np.asarray(img,'float32')
    img=cv2.resize(img,(240,240))
    img=np.transpose(img,(1,2,0))
    img=cv2.resize(img,(240,240))
    img=np.transpose(img,(2,0,1))
    img=np.asarray(img,'int')
    img=np.asarray(img,'float32')
    img1=img
#    print(np.shape(img))

    for j in range(img.shape[0]):        
        if (np.sum(img1[j])>0):
            data_T1.append(img[j])
 

    X = sitk.ReadImage(T2[i])
    img = sitk.GetArrayFromImage(X)
    img=np.asarray(img,'float32')
    img=cv2.resize(img,(240,240))
    img=np.transpose(img,(1,2,0))
    img=cv2.resize(img,(240,240))
    img=np.transpose(img,(2,0,1))
    img1=img
 
#    print(np.shape(img))

    for j in range(img.shape[0]):        
        if (np.sum(img1[j])>0):
            data_T2.append(img[j])
 

    X = sitk.ReadImage(SG[i])
    img = sitk.GetArrayFromImage(X)
    img=np.asarray(img,'float32')
    img=cv2.resize(img,(240,240))
    img=np.transpose(img,(1,2,0))
    img=cv2.resize(img,(240,240))
    img=np.transpose(img,(2,0,1))
    img=np.asarray(img,'int')
    img=np.asarray(img,'float32')
#    print(np.shape(img))

    for j in range(img.shape[0]):        
        if (np.sum(img1[j])>0):
            data_SG.append(img[j])
    
    X = sitk.ReadImage(TCE[i])
    img = sitk.GetArrayFromImage(X)
    img=np.asarray(img,'float32')
    img=cv2.resize(img,(240,240))
    img=np.transpose(img,(1,2,0))
    img=cv2.resize(img,(240,240))
    img=np.transpose(img,(2,0,1))
    img=np.asarray(img,'int')
    img=np.asarray(img,'float32')
#    print(np.shape(img))

    for j in range(img.shape[0]):        
        if (np.sum(img1[j])>0):
            data_TCE.append(img[j])

In [0]:
data_F1 = np.array(data_F1)
data_T1 = np.array(data_T1)
data_SG = np.array(data_SG)
data_TCE = np.array(data_TCE)
data_T2 = np.array(data_T2)


In [0]:
name="mydata"
hf=h5py.File(name+'.h5','w')
hf.create_dataset('f1_data', data=data_F1)
hf.create_dataset('t1_data', data=data_T1)
hf.create_dataset('SG_data', data=data_SG)
hf.create_dataset('t2_data', data=data_T2)
hf.create_dataset('TCE_data', data=data_TCE)
hf.create_dataset('X_data', data=data_X)
hf.close()

In [0]:
def distort_imgs(data_F1,data_TCE,data_SG,data_T1,data_TCE):
    """ data augumentation """
    x1, x2, x3, x4,x5, y = data_F1,data_TCE,data_SG,data_T1,data_T2
    # x1, x2, x3, x4, y = tl.prepro.flip_axis_multi([x1, x2, x3, x4, y],  # previous without this, hard-dice=83.7
    #                         axis=0, is_random=True) # up down
    x1, x2, x3, x4,x5, y = tl.prepro.flip_axis_multi([x1, x2, x3, x4,x5, y],
                            axis=1, is_random=True) # left right
#    x1, x2, x3, x4,x5, y = tl.prepro.elastic_transform_multi([x1, x2, x3, x4, x5, y],
#                            alpha=720, sigma=24, is_random=True)
    x1, x2, x3, x4,x5, y = tl.prepro.rotation_multi([x1, x2, x3, x4,x5, y], rg=20,
                            is_random=True, fill_mode='constant', row_index=1, col_index=2, channel_index=0) # nearest, constant
#    x1, x2, x3, x4,x5, y = tl.prepro.shift_multi([x1, x2, x3, x4,x5, y], wrg=0.10,
#                            hrg=0.10, is_random=True, fill_mode='constant')
    x1, x2, x3, x4,x5, y = tl.prepro.shear_multi([x1, x2, x3, x4,x5, y], 0.05,
                            is_random=True, fill_mode='constant', row_index=1, col_index=2, channel_index=0)
#    x1, x2, x3, x4,x5, y = tl.prepro.zoom_multi([x1, x2, x3, x4,x5, y],
#                            zoom_range=[0.9, 1.1], is_random=True,
#                            fill_mode='constant')
    return x1, x2, x3, x4, x5, y

In [0]:
data_F11,data_TCE1,data_SG1,data_T11,data_T21 = distort_imgs(data_F1,data_TCE,data_SG,data_T1,data_T2)
data_F12,data_TCE2,data_SG2,data_T12,data_T22 = distort_imgs(data_F1,data_TCE,data_SG,data_T1,data_T2)

data_SG=np.concatenate((data_SG,data_SG1,data_SG2),axis=0)
data_F1=np.concatenate((data_F1,data_F11,data_F12),axis=0)
data_TCE=np.concatenate((data_TCE,data_TCE1,data_TCE2),axis=0)
data_T1=np.concatenate((data_T1,data_T11,data_T12),axis=0)
data_T2=np.concatenate((data_T2,data_T21,data_T22),axis=0)

del data_SG1, data_SG2, data_F11, data_F12, data_TCE1, data_TCE2, data_T11, data_T12, data_T21, data_T22


In [0]:
for i in range(len(data_F1)):
    data_SG[i]=(data_SG[i]-np.mean(data_SG[i])+0.00001)/(np.std(data_SG[i])+0.00001)
    data_F1[i]=(data_F1[i]-np.mean(data_F1[i])+0.00001)/(np.std(data_F1[i])+0.00001)
    data_TCE[i]=(data_TCE[i]-np.mean(data_TCE[i])+0.00001)/(np.std(data_TCE[i])+0.00001)
    data_T1[i]=(data_T1[i]-np.mean(data_T1[i])+0.00001)/(np.std(data_T1[i])+0.00001)
    data_T2[i]=(data_T2[i]-np.mean(data_T2[i])+0.00001)/(np.std(data_T2[i])+0.00001)
data_X=[]
data_Y=[]

for i in range(len(data_F1)):
    k1=data_SG[i]
    k1=k1[:,:,np.newaxis,]
    k2=data_F1[i]
    k2=k2[:,:,np.newaxis,]
    k3=data_TCE[i]
    k3=k3[:,:,np.newaxis,]
    k4=data_T1[i]
    k4=k4[:,:,np.newaxis,] 
    k5=data_T2[i]
    k5=k5[:,:,np.newaxis,] 
    k6=np.concatenate((k1,k2,k3,k4,k5),axis=-1)=
    if(np.sum(k6)!=0):
        data_X.append(k6)
        data_Y.append(data_OT[i])

data_X = np.array(data_X)
data_Y = np.array(data_Y)
name="mydata_aug"
hf=h5py.File(name+'.h5','w')
hf.create_dataset('f1_data', data=data_F1)
hf.create_dataset('t1_data', data=data_T1)
hf.create_dataset('DP_data', data=data_SG)
hf.create_dataset('t2_data', data=data_T2)
hf.create_dataset('G1_data', data=data_TCE)
hf.create_dataset('X_data', data=data_X)
hf.close()

In [0]:
data_Y=data_Y[:,:,:,np.newaxis,]
for i in range(len(data_Y)):
    res=np.where(data_Y[i,:,:,0]>0)
    for j in range(len(res[0])):
        data_Y[i,res[0][j],res[1][j],0]=1.0
#    res=np.where(data_Y[i,:,:,0]<0.5)
#    for j in range(len(res[0])):
#        data_Y[i,res[0][j],res[1][j],0]=0.0

del data_SG, data_TCE, data_F1,data_T1, data_T2            
#for i in range(6414):
#    print(np.max(data_Y[i,:,:]))

X_train, X_test, Y_train, Y_test = train_test_split(data_X, data_Y, test_size=0.20, random_state=42)
name="mydata_training_testing_result"
hf=h5py.File(name+'.h5','w')
hf.create_dataset('X_train', data=X_train)
hf.create_dataset('Y_train', data=Y_train)
hf.create_dataset('X_test', data=X_test)
hf.create_dataset('Y_test', data=Y_test)
#hf.create_dataset('pred_result', data=pred_result)
#hf.create_dataset('X_data', data=data_X)
hf.close()


In [0]:
unetmodel = unet()
with h5py.File('mydata_training_testing_result.h5','r') as hf:
    print(list(hf.keys()))
    X_test=hf['X_test'][:]
    Y_test=hf['Y_test'][:]
    Y_train=hf['Y_train'][:]
    X_train=hf['X_train'][:]
    hf.close()
#model_checkpoint = K.ModelCheckpoint('unet.hdf5', monitor='loss',verbose=1, save_best_only=False)
model_checkpoint = K.ModelCheckpoint(
        filepath="weights.{epoch:04d}.h5",
        monitor="loss",
        save_best_only=False)

unetmodel.load_weights('weights.0009.h5')
unetmodel.fit(X_train, Y_train, batch_size=3, epochs=50,validation_data=(X_test,Y_test), shuffle=True,callbacks=[model_checkpoint],verbose=1,initial_epoch=11)
unetmodel.load_weights('weights.0037.h5')
unetmodel.evaluate(x=X_test, y=Y_test,batch_size=16, verbose=1)
pred_result = unetmodel.predict(X_train,batch_size=16)

name="mydata_train_data_result"
hf=h5py.File(name+'.h5','w')
hf.create_dataset('X_train', data=X_train)
hf.create_dataset('Y_train', data=Y_train)
hf.create_dataset('pred_result', data=pred_result)
hf.close()

with h5py.File('mydata_testing_data_result.h5','r') as hf:
    print(list(hf.keys()))
    X_test=hf['X_test'][:]
    Y_test=hf['Y_test'][:]
    pred_result=hf['pred_result'][:]
    hf.close()
    